In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nbdev import *
%nbdev_default_export utils
%nbdev_default_class_level 3

Cells will be exported to geomechy.utils,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_export
import numpy as np
from scipy.sparse import coo_matrix
import torch
from scipy.io import loadmat

# Utilities
> all utilities routines

## Dimentions and Units

### Metric Prefixes

In [ ]:
%nbdev_export
pico  = 1.e-12
nano  = 1.e-9
micro = 1.e-6
milli = 1.e-3
centi = 1.e-2
deci  = 1.e-1
kilo  = 1.e+3
mega  = 1.e+6
giga  = 1.e+9
tera  = 1.e+12

### Units

In [ ]:
%nbdev_export
units = {"distance":["meter", "ft", "inch"],
        "mass":["kg", "gram", "pound", "Ton"],
        "time":["second", "minute", "hour", "day", "year"],
        "temperature":["Kelvin", "Rankine", "Celsius", "Fahrenheit"],
        "volume":["litre", "gallon", "stb"],
        "force":["Newton", "dyne", "lbf"],
        "press":["kg", "gram", "pound", "Ton"],
        "energy":["Joules", "btu", "Cal"],
        "dinamicViscosity":["centiPoise", "Poise"],
        "kinematicViscosity":["centiStoke", "Stoke"],
        "absolutePermeability":["milliDarcy","Darcy"],
        "hydraulicConductivity":["inch_second","mm_second "],
        "dimensionless":["dimless"]}

### Distance

In [ ]:
%nbdev_export
meter = 1.                #meters -> meters
ft    = 0.3048*meter      #foots  -> meters
inch  = 2.54*centi*meter  #inches -> meters

### Mass

In [ ]:
%nbdev_export
kg     = 1.              #kilograms  -> kilograms
gram   = 1.*kg           #grams      -> kilograms
pound  = 0.45359237*kg   #pounds     -> kilograms
Ton    = 1000.*kg        #Tons       -> kilograms

### Time

In [ ]:
%nbdev_export
second  = 1.           #seconds -> seconds
minute  = 60.*second   #minutes -> seconds
hour    = 60.*minute   #hours   -> seconds
day     = 24.*hour     #days    -> seconds
year    = 365.25*day   #years   -> seconds

### Temperature

In [ ]:
%nbdev_export
Kelvin     = 1                             #kelvin     -> kelvin
Rankine    = 5./9.                         #rankine    -> kelvin
Celsius    = lambda C: C+273.15            #celsius    -> kelvin
Fahrenheit = lambda F: (F+459.67)*Rankine  #fahrenheit -> kelvin

### Volume

In [ ]:
%nbdev_export
litre  = (deci*meter)**3  #litre            -> cubic meter
gallon = 231*inch**3      #US gallons       -> cubic meter
stb    = 42*gallon        #standard barrels -> cubic meter

### Force

In [ ]:
%nbdev_export
Newton = 1.             #Newtons      -> Newtons
dyne   = 1.e-5*Newton   #dynes        -> Newtons
lbf    = 9.80665*pound  #pound-force  -> Newtons

### Press

In [ ]:
%nbdev_export
Pascal = 1.              #Pascals -> Pascals
atm    = 101325.*Pascal  #Pascals -> Atmosphères
bar    = 1e5*Pascal      #Pascals -> Bars
psi    = lbf/inch**2     #Pascals -> Psi

### Energy

In [ ]:
%nbdev_export
Joules   = 1.             #Joules -> Joules
btu     = 1054.06*Joules  #British thermal unit -> Joules
Cal     = 4184.*Joules    #Calories -> Joules

### Dinamic Viscosity

In [ ]:
%nbdev_export
centiPoise = milli*Pascal*second  #centiPoise -> Pascal*second
Poise      = 100*centiPoise       #Poise      -> Pascal*second

### Kinematic Viscosity

In [ ]:
%nbdev_export
centiStokes = 1.E-6*(meter**2)/second  #centiStokes -> square-meter per second
Stokes      = 1.E-4*(meter**2)/second  #centipoise  -> square-meter per second

### Absolute Permeability

In [ ]:
%nbdev_export
milliDarcy = 9.869233e-16*(meter**2)  #milliDarcy -> square-meter
Darcy      = 9.869233e-13*(meter**2)  #Darcy      -> square-meter

### Hydraulic Conductivity

In [ ]:
%nbdev_export
inch_second = inch*second         #inches per second      -> meter per second
mm_second   = milli*meter*second  #millimeters per second -> meter per second

### Dimensionless

In [ ]:
%nbdev_export
dimless = 1.

## Functions

### Lanczos Factorization

In [ ]:
%nbdev_export
def lanczos(A,xo,m,reorthog=0):
    p, n = A.shape
    assert p == n
    assert m <= n
    
    device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    xo      = xo.type(torch.float64).to(device)
    A       = A.type(torch.float64).to(device)
    Q       = torch.zeros(n,m+1, dtype=torch.float64, device=device)
    T       = torch.zeros(m+1,m, dtype=torch.float64, device=device)
    alpha   = torch.zeros(m,1, dtype=torch.float64, device=device)
    beta    = torch.zeros(m,1, dtype=torch.float64, device=device)
    q       = xo/torch.norm(xo)
    Q[:,0]  = q.reshape((-1))
    
    for k in range(m):
        w = torch.matmul(A,q)
        alpha[k] = torch.matmul(q.T,w)
        
        if k == 0:
            w = w - alpha[0]*q
        else:
            w = w - alpha[k]*q - beta[k-1]*Q[:,k-1].reshape((-1,1))
        if reorthog == 1:
            for i in range(1,k):
                h = torch.matmul(Q[:,i].reshape((-1,1)).type(torch.float64).T,w)
                w = w - Q[:,i].reshape((-1,1))*h
        
        beta[k] = torch.norm(w)
         
        if beta[k]<1e-20:
            return Q.cpu(), T.cpu()
        
        q = w/beta[k]
        Q[:,k+1] = q.reshape((-1))
        #print(q)
    T[:m,:] = torch.diag(beta[0:m-1].T[0],-1) + torch.diag(alpha.T[0]) + torch.diag(beta[0:m-1].T[0],1)
    T[m,m-1] = beta[-1]
    
    return Q.cpu(), T.cpu()

### Arnoldi Factorization

In [ ]:
%nbdev_export
def arnoldi(A,xo,m,reorthog=0):
    p, n = A.shape
    assert p == n
    assert m <= n
    
    
    device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    xo      = xo.type(torch.float64).to(device)
    A       = A.type(torch.float64).to(device)
    Q       = torch.zeros(n,m+1, dtype=torch.float64, device=device)
    H       = torch.zeros(m+1,m, dtype=torch.float64, device=device)
    tol     = n*2e-16;
    q       = xo/torch.norm(xo)
    Q[:,0]  = q.reshape((-1))
    
    for k in range(m):
        w  = A@Q[:,k]
        ow = torch.norm(w)
            
        for j in range(k+1):
            H[j,k] = Q[:,j].T@w
            w = w - H[j,k]*Q[:,j]

        if reorthog == 1:
            pass

        H[k+1,k] = torch.norm(w)

        if H[k+1,k] <= tol*ow:
            m = k
            H = H[:m+1,:m]
            Q = Q[:n,:m+1]
            return Q.cpu(), H.cpu(), m

        Q[:,k+1] = w/H[k+1,k]
    
    return Q.cpu(), H.cpu(), m

In [ ]:
%nbdev_hide
notebook2script()

Converted 00_core.ipynb.
Converted 01_physics.ipynb.
Converted 02_constitutive.ipynb.
Converted 03_shape.ipynb.
Converted 04_io.ipynb.
Converted 05_Solvers.ipynb.
Converted 06_materials.ipynb.
Converted 07_fem.ipynb.
Converted 08_Modules.ipynb.
Converted 09_base.ipynb.
Converted 10_utils.ipynb.
Converted index.ipynb.
Converted test.ipynb.
